# Federal Reserve Sentiment Analysis via Twitter's Natural Language Processing (NLP)

### A relatively newer trend in text analysis goes beyond topic detection and attempts to identify the emotion behind a text. This is called sentiment analysis, or also opinion mining and emotion AI.

Sentiment Analysis is the process of 'computationally' determining whether a text is positive, negative or neutral. It is also known as opinion mining, deriving the opinion or attitude of a speaker.

We will use Python and a Twitter Developer account to get the data regarding the latest Federal Reserve posts.

The Federal Reserve System (the FED, also translated as the Federal Reserve System) is an American institution responsible for supervising the banking system and defining the monetary policy applied in the country.

Their decisions influence not only the US economy but the entire world.

We start by importing the Libraries:

In [1]:
#Imports
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from googletrans import Translator
from unidecode import unidecode
import seaborn as sns

#### Authentication:
To fetch tweets through the Twitter API, you need to register an application through your twitter account. Follow these steps for the same:

Open this link and click the button: ‘Create New App’
Fill in the application data. You can leave the callback url field empty.
Once the app is created, you will be redirected to the app page.
Open the ‘Access Keys and Tokens’ tab.
Copy 'Consumer Key', 'Consumer Secret', 'Access Token' and 'Access Token Secret'.
Just below you will include the copied keys:

In [ ]:
consumerKey = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
consumerSecret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
accessToken = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
accessTokenSecret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

#### We will then create the authentication:

In [ ]:
auth = tweepy.OAuthHandler(consumerKey,consumerSecret)
#"Directing" the access token
auth.set_access_token(accessToken, accessTokenSecret) 
#Creating API object using our credentials
api = tweepy.API(auth, wait_on_rate_limit = True)

#### Here we will define the account to be analyzed (federalreserve) and the language:

In [ ]:
name_id = 'federalreserve'
language = 'en'

#### Now we will scrape the last 10 tweets:

In [ ]:
posts = api.user_timeline(screen_name=name_id,
                          count = 10,
                          lang =language,
                          tweet_mode="extended")

#### We are going to use List Comprehension which allows you to create and process lists cleanly and easily to insert into the dataframe.

In [ ]:
df_tweets=pd.DataFrame([tweet.full_text for tweet in posts], columns=['Tweets'])
df_tweets.head()

#### Defining the cleaning function:

In [ ]:
def limpando_chars(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #removing @
    text = re.sub('#', '', text) #removing #
    text = re.sub('RT[\s]+', '', text) #removing RT
    text = re.sub('https?:\/\/\S+', '', text) #removing hyperlink
    text = re.sub('&amp','', text) #removing HTML markup from start
    return text

#### Running the cleanup:

In [ ]:
df_tweets['Tweets'] = df_tweets['Tweets'].apply(limpando_chars)

#### Defining Subjectivity and Polarity:

In [ ]:
# Function that analyzes and obtains subjectivity
def capture_subjetividade_pt(text):
    return TextBlob(text).sentiment.subjectivity
# Function that analyzes and obtains polarity
def capture_polaridade_pt(text):
    return  TextBlob(text).sentiment.polarity
def translate_text(text):
    return Translator().translate(unidecode(text)).text
#translating
if language!= 'en':
    df_tweets['Tweets'] = df_tweets['Tweets'].apply(translate_text)
else:
    pass
# Create two columns of subjectivity and polarity
df_tweets['Subjetividade'] = df_tweets['Tweets'].apply(capture_polaridade_pt)
df_tweets['Polaridade'] = df_tweets['Tweets'].apply(capture_subjetividade_pt)
# Shows a new dataframe with the subjectivity and polarity columns
df_tweets.tail()

##Code based on the original by the author Germano Yoneda

#### Creating the Analysis generation function:

In [ ]:
def gerar_analise(score):
    if score < 0:
        return 'Negativo'
    elif score == 0:
        return 'Neutro'
    else:
        return 'Positivo'
df_tweets['Analise'] = df_tweets['Polaridade'].apply(gerar_analise)

#### We then store the result in a Dataset:

In [ ]:
resultado = pd.DataFrame(df_tweets['Analise'].value_counts())

#### Visualizing the result through a graph:

In [ ]:
plt.figure()
ax = sns.barplot(x=resultado['Analise'], y=df.index, data=df_tweets)
ax.set_xlabel('Number of tweets')
ax.set_ylabel('Sentiment')

#### This approach is very interesting especially if applied to real-time Dashboards and combined with other analyses. This is just a study of the tool's possibilities.
Always remembering: This is just a preliminary high risk trading strategy study for information purposes only. In any strategy, the following must always be considered: Risk Management, Operating Costs, Risk Tolerance Profile, etc. This article is by no means an investment recommendation.

— — — — — — — — — — — — — — — — — — — — — — — — —

Sources:

infoq.com

geeksforgeeks.org

Germano Yoneda

maisretorno.com